# Tutorial: Integrate Neuroscience Datasets from Multiple Sources using MINDS

## Initialize and configure

nexus forge - interface, to manipulate knowledge graph, query data etc.

neurom[plotly] - package for visualizing neurons 

allensdk - package to access, download and easily manipulate data from the Allen Institute 

In [ ]:
!pip install nexusforge==0.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 720 kB 14.6 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 1.1 MB 60.5 MB/s 
     |████████████████████████████████| 482 kB 61.4 MB/s 
     |████████████████████████████████| 70 kB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 50.4 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
     |████████████████████████████████| 385 kB 67.3 MB/s 
     |████████████████████████████████| 57 kB 5.9 MB/s 
     |████████████████████████████████| 41 kB 758 kB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 76.6 MB/s 
     |████████████████████████████████| 144 kB 77.1 MB/s 
     |████████████████████████████████| 99 kB 9

In [ ]:
!pip install allensdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.2 MB/s 
     |████████████████████████████████| 3.0 MB 56.6 MB/s 
     |████████████████████████████████| 3.4 MB 41.6 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 1.3 MB 41.3 MB/s 
     |████████████████████████████████| 48.4 MB 26 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 130 kB 67.9 MB/s 
     |████████████████████████████████| 118 kB 60.7 MB/s 
     |████████████████████████████████| 185 kB 48.5 MB/s 
     |████████████████████████████████| 4.3 MB 59.8 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 130 kB 61.0 MB/s 
     |████████████████████████████████| 26.5 MB 1.4 MB/s 
     |████████████████████

In [ ]:
!pip install neurom[plotly]==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 474 kB 15.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.9 MB 55.7 MB/s 
     |████████████████████████████████| 281 kB 64.0 MB/s 
  Created wheel for neurom: filename=neurom-3.0.1-py3-none-any.whl size=100039 sha256=49580d9c65a01271eff9f93ba3a6a5c893c15e7e5a356a54137d5bc1468cdf9f
  Stored in directory: /root/.cache/pip/wheels/d3/47/0c/ad39d56ccedfb07bed19bde0dc51f2beafa786033117a036f3
Successfully built neurom
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
!pip install --upgrade nest-asyncio==1.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.5
    Uninstalling nest-asyncio-1.5.5:
      Successfully uninstalled nest-asyncio-1.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.6 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
nbclient 0.6.6 requires traitlets>=5.2.2, but you have traitlets 5.1.1 which is incompatible.


### Get an authentication token

The [Nexus sandbox application](https://sandbox.bluebrainnexus.io) can be used to get a token:

- Step 1: From the [web page](https://sandbox.bluebrainnexus.io), click on the login button in the top right corner and follow the instructions on screen.

- Step 2: You will then see a `Copy token` button in the top right corner. Click on it to copy the token to the clipboard.


Once a token is obtained, proceed to paste it as the value of the `TOKEN` variable below.

__Important__: A Nexus token is valid for 8 hours, if your working session is open for more than 8 hours, you may need to refresh the value of the token and reintialize the forge client in the _'Configure a forge client to store, manage and access datasets'_ section below.

A token is necessary to run code on the sandbox for nexus or the online virtual environment instead of doing it on your local. Running and initiating on the local computer or organiztion is possible but for trials only, it can be done via the sandbox. 

In [ ]:
import getpass

In [ ]:
TOKEN = getpass.getpass()

··········


### Configure a forge client to store, manage and access datasets

In [ ]:
import uuid
import base64
import requests
import json
from pathlib import Path

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.mappings import DictionaryMapping

from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import CellTypesCache

r - json file containing the mapping supported by Nexus 

Next set of lines create a copy of this file in your directory. The next 2 code sections creates a "bucket" based on your account in Nexus (within github-users organisation for MOOP edx course) 

Essentially, this connects your account from nexus to the code/s being run in this notebook and allows data to be stored within nexus. 

In [ ]:
r = requests.get('https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/rdfmodel/jsonldcontext.json')
dirpath = './rdfmodel'
Path(dirpath).mkdir(parents=True, exist_ok=True)
with open(f'{dirpath}/jsonldcontext.json', 'w') as outfile:
    json.dump(r.json(), outfile)

In [ ]:
ORG = "github-users"
PROJECT = "cdjcodes"  # Provide here the automatically created project name created when you logged into the Nexus sandbox instance.

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/forge.yml",
                            bucket=f"{ORG}/{PROJECT}",
                            endpoint="https://sandbox.bluebrainnexus.io/v1",
                            token=TOKEN)

## Download datasets from Allen Cell Types Database and MouseLight

### Download mouse neuron morphologies from the Allen Cell Types Database

We will be downloading mouse neuron morphology data from the [Allen Cell Types Database](https://celltypes.brain-map.org/). The [AllenSDK](https://allensdk.readthedocs.io/en/latest/) can be used for data download.

In [ ]:
ALLEN_DIR = "allen_cell_types_database" #type of database to access

In [ ]:
ctc = CellTypesCache(manifest_file=f"{ALLEN_DIR}/manifest.json")

In [ ]:
MAX_CELLS = 10 # Increase to include more cells
SPECIES = CellTypesApi.MOUSE

In [ ]:
#Get the identifiers for selected cells given species
nm_allen_identifiers = [cell["id"] for cell in ctc.get_cells(species=[SPECIES], 
                      require_reconstruction = True)][:MAX_CELLS]
print(f"Selected a mouse neuron with identifier: {nm_allen_identifiers}")

Selected a mouse neuron with identifier: [485909730, 323865917, 476135066, 502614426, 480087928, 473611755, 475549334, 501799874, 314804042, 555241040]


Select metadata

In [ ]:
with open(f"{ALLEN_DIR}/cells.json") as f:
    allen_cell_types_metadata = json.load(f)

In [ ]:
nm_allen_metadata = [neuron for neuron in allen_cell_types_metadata if neuron["specimen__id"] in nm_allen_identifiers]

Download reconstruction files

In [ ]:
for identifier in nm_allen_identifiers:
    ctc.get_reconstruction(identifier)

2022-07-16 06:51:22,991 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/500961530
2022-07-16 06:51:23,819 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/492413274
2022-07-16 06:51:24,948 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/512328422
2022-07-16 06:51:25,979 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/519262719
2022-07-16 06:51:27,072 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491770972
2022-07-16 06:51:27,908 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491253838
2022-07-16 06:51:28,740 allensdk.api.api.retrieve_fi

### Download mouse neuron electrophysiology recordings from the Allen Cell Types Database

Download Electrophysiology recordings

In [ ]:
for identifier in nm_allen_identifiers:
    ctc.get_ephys_data(identifier)

2022-07-16 06:51:36,010 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491316386
2022-07-16 06:51:41,575 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491200673
2022-07-16 06:51:46,026 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491316017
2022-07-16 06:51:55,842 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/544847110
2022-07-16 06:52:01,287 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491198827
2022-07-16 06:52:26,815 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/496389059
2022-07-16 06:52:38,695 allensdk.api.api.retrieve_fi

### Download mouse neuron morphologies from MouseLight project

We will be downloading mouse neuron morphology data from the [MouseLight project](https://www.janelia.org/project-team/mouselight).

In [ ]:
URL_GRAPHQL = "http://ml-neuronbrowser.janelia.org/graphql/"

In [ ]:
URL_JSON = "http://ml-neuronbrowser.janelia.org/export"

In [ ]:
URL_SWC = "http://ml-neuronbrowser.janelia.org/export"

Select metadata

In [ ]:
nm_request = requests.post(URL_GRAPHQL, json={"operationName":"SearchNeurons",
                                "variables":{
                                    "context":{
                                        "scope":6,
                                        "nonce":"ckzmxfe9600033e68qnwoa59t",
                                        "ccfVersion": "CCFV25",
                                        "predicates":[{
                                            "predicateType":"ANATOMICAL",
                                            "tracingIdsOrDOIs":["1"],
                                            "tracingIdsOrDOIsExactMatch":False,
                                            "tracingStructureIds":["68e76074-1777-42b6-bbf9-93a6a5f02fa4"],
                                            "nodeStructureIds":["c37953e1-a1e9-4b9a-847e-08d9566ced65"],
                                            "operatorId":None,
                                            "amount":0,
                                            "brainAreaIds":[],
                                            "arbCenter":{
                                                "x":None,
                                                "y":None,
                                                "z":None},
                                            "arbSize":None,
                                            "invert":False,
                                            "composition":3
                                        }]
                                    }
                                },
                                "query":"""query SearchNeurons($context: SearchContext) {\n  searchNeurons(context: $context) 
                                            {\n    totalCount\n    queryTime\n    nonce\n    \n    neurons {\n      id\n      
                                            idString\n      tracings {\n        id\n        tracingStructure {\n          id\n          
                                            name\n          value\n          __typename\n        }\n        soma {\n          id\n          
                                            x\n          y\n          z\n          radius\n          parentNumber\n          
                                            sampleNumber\n          brainAreaIdCcfV30\n          structureIdentifierId\n          
                                            __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    
                                            __typename\n  }\n}\n"""
                               })
nm_mouselight_graphql = json.loads(nm_request.text)["data"]["searchNeurons"]["neurons"]
nm_mouselight_names = [x["idString"] for x in nm_mouselight_graphql]

In [ ]:
nm_mouselight_metadata = list()
for name in nm_mouselight_names[0:MAX_CELLS]:
    a = requests.post(URL_JSON, 
                          json={"ids": [name],"format":1},
                          headers={"Accept": "*/*", "Content-Type": "application/json"})
    #print(a.content.decode('utf-8'))
    c = json.loads(a.content.decode('utf-8'))
    nm_mouselight_metadata.append(c["contents"])

In [ ]:
for i, nm in enumerate(nm_mouselight_metadata):
    allenId = nm["neurons"][0]["soma"]["allenId"]
    allenInfo = nm["neurons"][0]["allenInformation"]
    for info in allenInfo:
        if info["allenId"] == allenId:
            allenLabel = info["name"]
    nm_mouselight_metadata[i]["neurons"][0]["allenLabel"] = allenLabel

Download reconstruction files

In [ ]:
for name in nm_mouselight_names[0:MAX_CELLS]:
    a = requests.post(URL_SWC, 
                      json={"ids": [name],"format":0},
                      headers={"Accept": "*/*", "Content-Type": "application/json"})
    c = eval(a.content)
    base64_message = c["contents"]
    base64_bytes = base64_message.encode('ascii')
    message_bytes = base64.b64decode(base64_bytes)
    dirpath = './mouselight'
    Path(dirpath).mkdir(parents=True, exist_ok=True)
    with open(f"{dirpath}/{name}.swc", "wb") as f:
        f.write(message_bytes)

## Mapping

### Map the Allen Cell Types Database neuron morphologies to Neuroshapes

In [ ]:
allen_nm_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/allen_morphology_dataset.hjson") # TODO
nm_allen_resources = forge.map(nm_allen_metadata, allen_nm_mapping)

### Map the Allen Cell Types Database neuron electrophysiology recordings to Neuroshapes

In [ ]:
allen_ephys_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/allen_ephys_dataset.hjson") # TODO
nephys_allen_resources = forge.map(nm_allen_metadata, allen_ephys_mapping)

### Map the MouseLight neuron morphologies to Neuroshapes

In [ ]:
mouselight_nm_mapping = DictionaryMapping.load("https://raw.githubusercontent.com/BlueBrain/nexus/ef830192d4e7bb95f9351c4bdab7b0114c27e2f0/docs/src/main/paradox/docs/getting-started/notebooks/mappings/mouselight_dataset.hjson") # TODO
nm_mouselight_resources = forge.map(nm_mouselight_metadata, mouselight_nm_mapping)

## Register

If the registration fails, try refreshing the access token and reinitializing the forge client in the _'Configure a forge client to store, manage and access datasets'_ section.

### Register the Allen Cell Types Database neuron morphologies

In [ ]:
for resource in nm_allen_resources:
    resource.id = forge.format("identifier", "neuronmorphologies", str(uuid.uuid4()))

In [ ]:
forge.register(nm_allen_resources)

<count> 10
<action> _register_many
<succeeded> True


### Register the Allen Cell Types Database neuron electrophysiology recordings

In [ ]:
for resource in nephys_allen_resources:
    resource.id = forge.format("identifier", "traces", str(uuid.uuid4()))

In [ ]:
forge.register(nephys_allen_resources)

<count> 10
<action> _register_many
<succeeded> True


### Register the MouseLight neuron morphologies

In [ ]:
for resource in nm_mouselight_resources:
    resource.id = forge.format("identifier", "neuronmorphologies", str(uuid.uuid4()))

In [ ]:
forge.register(nm_mouselight_resources)

<count> 10
<action> _register_many
<succeeded> True


### Save the created resources in JSON files

In [ ]:
dirpath = './database'
Path(dirpath).mkdir(parents=True, exist_ok=True)
with open(f"{dirpath}/mouselight-protocols.json","w") as f:
    json.dump(forge.as_jsonld(nm_mouselight_resources, form="expanded"),f)

In [ ]:
with open(f"{dirpath}/allen-morphologies-protocols.json","w") as f:
    json.dump(forge.as_jsonld(nm_allen_resources, form="expanded"),f)

In [ ]:
with open(f"{dirpath}/allen-ephys-protocols.json","w") as f:
    json.dump(forge.as_jsonld(nephys_allen_resources, form="expanded"),f)

## Access

### Set filters

In [ ]:
_type = "NeuronMorphology"

filters = {"type": _type}

### Run Query

In [ ]:
number_of_results = 10  # You can limit the number of results, pass `None` to fetch all the results

data = forge.search(filters, limit=number_of_results)

print(f"{str(len(data))} dataset(s) of type {_type} found")

10 dataset(s) of type NeuronMorphology found


### Display the results as pandas dataframe

In [ ]:
property_to_display = ["id","name","subject","brainLocation.brainRegion.id","brainLocation.brainRegion.label","brainLocation.layer.id","brainLocation.layer.label", "contribution","brainLocation.layer.id","brainLocation.layer.label","distribution.name","distribution.contentUrl","distribution.encodingFormat"]
reshaped_data = forge.reshape(data, keep=property_to_display)

forge.as_dataframe(reshaped_data)

,id,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.layer,contribution.type,contribution.agent.id,contribution.agent.type,contribution.agent.label,distribution.contentUrl,distribution.encodingFormat,...,name,subject.type,subject.age.period,subject.identifier,subject.name,subject.species.label,subject.strain.label,subject.age.unitCode,subject.age.value,subject.sex.label
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp5,5,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Cux2-CreERT2;Ai14-205530.03.02.01,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2,NaN,NaN,NaN
1,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISpl4,4,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Rorb-IRES2-Cre-D;Ai14-230822.04.02.01,Subject,Post-natal,502081962,Rorb-IRES2-Cre-D;Ai14-230822,Mus musculus,Rorb-IRES2-Cre,,,
2,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp5,5,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Cux2-CreERT2;Ai14-205530.03.02.01,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2,,,
3,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp4,4,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Rorb-IRES2-Cre-D;Ai14-197330.06.01.01,Subject,Post-natal,479695183,Rorb-IRES2-Cre-D;Ai14-197330,Mus musculus,Rorb-IRES2-Cre,,,
4,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp5,5,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Pvalb-IRES-Cre;Ai14-185362.03.01.01,Subject,Post-natal,475196647,Pvalb-IRES-Cre;Ai14(IVSCC)-185362,Mus musculus,Pvalb-IRES-Cre,,,
5,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure/33,VISp6a,6a,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Ctgf-2A-dgCre;Ai14(IVSCC)-229233.03.02.01,Subject,Post-natal,501715368,Ctgf-2A-dgCre;Ai14(IVSCC)-229233,Mus musculus,Ctgf-T2A-dgCre,,,
6,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp5,5,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Scnn1a-Tg3-Cre;Ai14-172530.06.01.01,Subject,Post-natal,322489588,Scnn1a-Tg3-Cre;Ai14(GSL)-172530,Mus musculus,Scnn1a-Tg3-Cre,,,
7,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp4,4,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Scnn1a-Tg3-Cre;Ai14-187849.06.01.01,Subject,Post-natal,475849748,Scnn1a-Tg3-Cre;Ai14(IVSCC)-187849,Mus musculus,Scnn1a-Tg3-Cre,,,
8,https://bbp.epfl.ch/neurosciencegraph/data/neu...,http://api.brain-map.org/api/v2/data/Structure...,VISp5,5,Contribution,https://www.grid.ac/institutes/grid.417881.3,Organization,Allen Institute for Brain Science,https://sandbox.bluebrainnexus.io/v1/files/git...,application/swc,...,Rorb-IRES2-Cr

### Download

In [ ]:
dirpath = "./downloaded/"
forge.download(data, "distribution.contentUrl", dirpath)

In [ ]:
ls ./downloaded/

reconstruction.swc


### Display a result as 3D Neuron Morphology

In [ ]:
from neurom import load_morphology
from neurom.view.plotly_impl import plot_morph3d
import IPython

In [ ]:
neuron = load_morphology(f"{dirpath}/{data[0].distribution.name}")
plot_morph3d(neuron, inline=False)
IPython.display.HTML(filename='./morphology-3D.html')

Output hidden; open in https://colab.research.google.com to view.

## Version the dataset
Tagging a dataset is equivalent to `git tag`. It allows to version a dataset.

In [ ]:
forge.tag(data, value="releaseV112")

<count> 10
<action> _tag_many
<succeeded> True


In [ ]:
# The version argument can be specified to retrieve the dataset at a given tag.

tagged_data = forge.retrieve(id=data[0].id, version="releaseV112")

In [ ]:
forge.as_dataframe(tagged_data)

,id,type,_schemaProject,apicalDendrite,brainLocation.type,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.coordinatesInBrainAtlas.valueX,brainLocation.coordinatesInBrainAtlas.valueY,brainLocation.coordinatesInBrainAtlas.valueZ,...,objectOfStudy.id,objectOfStudy.type,objectOfStudy.label,subject.type,subject.age.period,subject.identifier,subject.name,subject.species.label,subject.strain.label,tag__apical
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,"[Dataset, NeuronMorphology]",https://sandbox.bluebrainnexus.io/v1/projects/...,spiny,BrainLocation,http://api.brain-map.org/api/v2/data/Structure...,VISp5,8881.0,953.839501,7768.226958,...,http://bbp.epfl.ch/neurosciencegraph/taxonomie...,ObjectOfStudy,Single Cell,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2,intact


In [ ]:
data[0].description="Neuron Morphology from Allen"

In [ ]:
forge.update(data[0])

<action> _update_one
<succeeded> True


In [ ]:
non_tagged_data = forge.retrieve(id=data[0].id)

In [ ]:
forge.as_dataframe(non_tagged_data)

,id,type,_schemaProject,apicalDendrite,brainLocation.type,brainLocation.brainRegion.id,brainLocation.brainRegion.label,brainLocation.coordinatesInBrainAtlas.valueX,brainLocation.coordinatesInBrainAtlas.valueY,brainLocation.coordinatesInBrainAtlas.valueZ,...,objectOfStudy.id,objectOfStudy.type,objectOfStudy.label,subject.type,subject.age.period,subject.identifier,subject.name,subject.species.label,subject.strain.label,tag__apical
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,"[Dataset, NeuronMorphology]",https://sandbox.bluebrainnexus.io/v1/projects/...,spiny,BrainLocation,http://api.brain-map.org/api/v2/data/Structure...,VISp5,8881.0,953.839501,7768.226958,...,http://bbp.epfl.ch/neurosciencegraph/taxonomie...,ObjectOfStudy,Single Cell,Subject,Post-natal,485250100,Cux2-CreERT2;Ai14-205530,Mus musculus,Cux2-CreERT2,intact
